**Import libraries and Functions**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import sqlite3

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

import tensorflow as tf

In [ ]:
tf.__version__

'2.6.0'

Inisialisasi Benih

Hal ini penting untuk memastikan bahwa hasil yang kita capai dari model ini dapat dicapai kembali dengan tepat.  Ini memastikan bahwa proses stokastik pelatihan model jaringan saraf dapat direproduksi.

In [ ]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

**Load the dataset**

Di sini, kita akan memuat data dari database sqlite menggunakan pandas.  Atau, kami juga dapat memuat data dari csv.  Juga, dari sklearn.datasets juga

In [ ]:
connection = sqlite3.connect('database.sqlite')
data = pd.read_sql_query(''' SELECT * FROM IRIS ''', connection)
print("Shape of data: {}".format(data.shape))

Shape of data: (150, 6)


Mencetak 3 baris pertama dari database akan memberikan pemahaman tingkat yang lebih tinggi tentang "apa yang sebenarnya ada dalam data"

In [ ]:
data.head(3)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa


Memeriksa apakah dataset berisi nilai null/na atau tidak.

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


**Pengamatan :**

1. Tidak ada nilai nol dalam kumpulan data.
2. Jumlah total pengamatan adalah 150.
3. Semua fitur kecuali fitur keluaran yaitu Spesies bertipe float.

In [ ]:
Y = data['Species']
X = data.drop(['Id', 'Species'], axis=1)
print("Shape of Input  features: {}".format(X.shape))
print("Shape of Output features: {}".format(Y.shape))

Shape of Input  features: (150, 4)
Shape of Output features: (150,)


**Encoding the Output/Response Variable**

In [ ]:
Y.value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

In [ ]:
lbl_clf = LabelEncoder()
Y_encoded = lbl_clf.fit_transform(Y)

#Keras requires your output feature to be one-hot encoded values.
Y_final = tf.keras.utils.to_categorical(Y_encoded)

print("Therefore, our final shape of output feature will be {}".format(Y_final.shape))

Therefore, our final shape of output feature will be (150, 3)


**Splitting the dataset in 75-25 ratio**

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y_final, test_size=0.25, random_state=seed, stratify=Y_encoded, shuffle=True)

print("Training Input shape\t: {}".format(x_train.shape))
print("Testing Input shape\t: {}".format(x_test.shape))
print("Training Output shape\t: {}".format(y_train.shape))
print("Testing Output shape\t: {}".format(y_test.shape))

Training Input shape	: (112, 4)
Testing Input shape	: (38, 4)
Training Output shape	: (112, 3)
Testing Output shape	: (38, 3)


**Standardizing the dataset**

In [ ]:
std_clf = StandardScaler()
x_train_new = std_clf.fit_transform(x_train)
x_test_new = std_clf.transform(x_test)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(10, input_dim=4, activation=tf.nn.relu, kernel_initializer='he_normal', 
                                kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(7, activation=tf.nn.relu, kernel_initializer='he_normal', 
                                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(5, activation=tf.nn.relu, kernel_initializer='he_normal', 
                                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)))
model.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

iris_model = model.fit(x_train_new, y_train, epochs=700, batch_size=7)

Epoch 1/700
16/16 [==============================] - 1s 2ms/step - loss: 1.5452 - accuracy: 0.3482
Epoch 2/700
16/16 [==============================] - 0s 2ms/step - loss: 1.5885 - accuracy: 0.2768
Epoch 3/700
16/16 [==============================] - 0s 2ms/step - loss: 1.3592 - accuracy: 0.4107
Epoch 4/700
16/16 [==============================] - 0s 2ms/step - loss: 1.3648 - accuracy: 0.3661
Epoch 5/700
16/16 [==============================] - 0s 2ms/step - loss: 1.2919 - accuracy: 0.3571
Epoch 6/700
16/16 [==============================] - 0s 2ms/step - loss: 1.3925 - accuracy: 0.3929
Epoch 7/700
16/16 [==============================] - 0s 2ms/step - loss: 1.2855 - accuracy: 0.4643
Epoch 8/700
16/16 [==============================] - 0s 2ms/step - loss: 1.2647 - accuracy: 0.4821
Epoch 9/700
16/16 [==============================] - 0s 2ms/step - loss: 1.3043 - accuracy: 0.4107
Epoch 10/700
16/16 [==============================] - 0s 2ms/step - loss: 1.2251 - accuracy: 0.4821
Epoch 11/

In [ ]:
model.evaluate(x_test_new, y_test)

2/2 [==============================] - 0s 7ms/step - loss: 0.0960 - accuracy: 1.0000


[0.09598061442375183, 1.0]